In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import matplotlib

In [43]:
matplotlib.rcParams.update({'font.size': 12})
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [51]:
def save_prediction_to_csv(y_pred):
    """
    Use this function to save your prediction result to a csv file.
    The resulting csv file is named as [team_name].csv

    :param y_pred: an array or a pandas series that follows the SAME index order as in the testing data
    """
    pd.DataFrame(dict(
        target=y_pred
    )).to_csv('predictions.csv', index=False, header=False)

In [69]:
def prepare_data(thresh = 0.85):

    #### Read Train and Test Data
    training_data = pd.read_csv('/home/rajneesh/Desktop/Queen_City_Hackathon/training.csv', index_col=0)
    testing_data = pd.read_csv('/home/rajneesh/Desktop/Queen_City_Hackathon/testing.csv', index_col=0)

    ### Check for the co relared columns 

    # Create correlation matrix
    corr_matrix = training_data.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find index of feature columns with correlation greater than 0.95
    columns_to_drop = [column for column in upper.columns if any(upper[column] > thresh)]
    if 'target' in columns_to_drop:
        columns_to_drop.remove('target')

    ### Drop all the co-related columns
    data_nodups_df = training_data.drop(columns=columns_to_drop)


    #### Data Preparation
    train_df = data_nodups_df.fillna(data_nodups_df.mean())

    #y = training_data['target']
    #X = data_nonull_df.drop(columns=['target'])

    test_nodups_df = testing_data.drop(columns=columns_to_drop)
    test_df = test_nodups_df.fillna(data_nodups_df.mean())
    
    return (train_df, test_df)

In [70]:
(train_df, test_df) = prepare_data(thresh=0.75)
y = train_df['target']
X = train_df.drop(columns = ['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=101)

In [71]:
pipe = Pipeline([('rforest', RandomForestRegressor())])

pipe.fit(X_train, y_train)
print("Test score: {:.2f}".format(pipe.score(X_test, y_test )))

Test score: 0.09


/home/rajneesh/miniconda3/envs/py36/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


***
### Train on full dataset

In [62]:
pipe.fit(X, y)
y_pred = pipe.predict(test_df)

save_prediction_to_csv(y_pred)

In [63]:
y_pred.sum()

2.1225714285099992